In [1]:
import yaml


In [7]:
main_path = r'../../..'
data_path = main_path+'/data'
import sys
sys.path.append(main_path)


In [11]:
from BayDS.lib.training import fast_auc

In [14]:
from sklearn.model_selection import KFold
import pandas as pd

In [73]:
working_folder = '../../../data/sub'
lb = yaml.load(open(f'{working_folder}/lb.yaml','r'),Loader=yaml.FullLoader)

In [74]:
target = pd.read_csv(f'{working_folder}/target.csv').set_index('TransactionID')

In [72]:
# st = pd.read_pickle(f'{working_folder}/undefined/stacked.pkl')
# st['oof'].shape
# stacked_oof = pd.DataFrame(index=target.index)
# stacked_oof['isFraud'] = st['oof']
# stacked_oof.to_csv(f'{working_folder}/stacked_oof.csv')

In [75]:
lb

[{'oofFile': 'oof_all_data_best_lgb_26.csv',
  'predictionFile': 'prediction_all_data_best_lgb_26.csv',
  'score': 0.9459},
 {'oofFile': 'oof_all_data_best_lgb_timefreq.csv',
  'predictionFile': 'prediction_all_data_best_lgb_timefreq.csv',
  'score': 0.9504},
 {'oofFile': 'oof_all_data_best_lgb.csv',
  'predictionFile': 'prediction_all_data_best_lgb.csv',
  'score': 0.9484},
 {'oofFile': 'oof_lgb_new_card_id_del_mean020_v3.csv',
  'predictionFile': 'lgb_new_card_id_del_mean020_v3.csv',
  'score': 0.947},
 {'oofFile': 'oof_best_submit_09469.csv',
  'predictionFile': 'submission_09469.csv',
  'score': 0.9469},
 {'oofFile': 'stacked_oof.csv',
  'predictionFile': 'stacked.csv',
  'score': 0.939}]

In [76]:
X = []
y = []
for isub, sub in enumerate(lb):
    print(f'*** {isub}: {sub["score"]} -- {sub["oofFile"]}'  )
    folds = KFold(n_splits=5)
    pfile = f"{working_folder}/{sub['oofFile']}"
    oof = pd.read_csv(pfile).set_index('TransactionID')

    splitter = folds.split(oof)
    oof_aucs = []      
    for ifold,(train_idx, val_idx) in enumerate(splitter):
    #     print(ifold,val_idx)
        val_idx_trans = oof.iloc[val_idx].index  
        val_x = oof['isFraud'].loc[val_idx_trans]
        val_y = target['isFraud'].loc[val_idx_trans]
        auc = fast_auc(val_y,val_x)
        print(auc)
        oof_aucs.append(auc)
    X.append(oof_aucs)
    y.append(sub['score'])
    

*** 0: 0.9459 -- oof_all_data_best_lgb_26.csv
0.9120146257737021
0.9445642303513123
0.9353198507224967
0.9523825555833152
0.9310722289065904
*** 1: 0.9504 -- oof_all_data_best_lgb_timefreq.csv
0.9200897137273757
0.9436437284124829
0.9408082617580837
0.956501557348579
0.93628500194496
*** 2: 0.9484 -- oof_all_data_best_lgb.csv
0.918912852526571
0.9419404308890484
0.9369641794849076
0.9553157368036542
0.9360824321466477
*** 3: 0.947 -- oof_lgb_new_card_id_del_mean020_v3.csv
0.9129049159819259
0.9423006368296207
0.9367266085128249
0.9544686181601808
0.932655036015369
*** 4: 0.9469 -- oof_best_submit_09469.csv
0.9212245167648406
0.9404617392759979
0.9366978794048867
0.9526967014398073
0.9301525414489835
*** 5: 0.939 -- stacked_oof.csv
0.933775394649654
0.9482903382075164
0.9429304633256416
0.9551325404552266
0.920039782700944


In [77]:
X = np.array(X)
y = np.array(y)

In [88]:
from sklearn.linear_model import LinearRegression,Ridge
lr = LinearRegression(fit_intercept = False)
lr.fit(X[:,:],y)
y_pred = lr.predict(X[:,:])
print(y,y_pred)

[0.9459 0.9504 0.9484 0.947  0.9469 0.939 ] [0.94578715 0.95108418 0.94946346 0.94665166 0.94522056 0.93938696]


In [89]:
lr.coef_

array([ 0.02316714,  0.11418089,  0.45246619, -0.43529977,  0.86800837])